In [1]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html
import boto3 
import datetime
import time

In [2]:
dx = boto3.client('dataexchange', region_name='us-east-1')

In [3]:
data_set_id='d0e9bd6148e8f14889980954017b0927'

In [4]:
dx.get_data_set(DataSetId='d0e9bd6148e8f14889980954017b0927')

{'ResponseMetadata': {'RequestId': '9be03daa-d578-4bc2-8509-9943ced0e83f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '474',
   'connection': 'keep-alive',
   'date': 'Thu, 12 Nov 2020 04:19:59 GMT',
   'x-amzn-requestid': '9be03daa-d578-4bc2-8509-9943ced0e83f',
   'x-amz-apigw-id': 'V4GtaHSioAMF2dw=',
   'x-amzn-trace-id': 'Root=1-5facb7ef-685a1c097a558fd0530df61a',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 a8dfbcfd2b9a376324907969cf0f5cdc.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'HIO50-C2',
   'x-amz-cf-id': 'oFKKlW_g_KVhu_US7WABD21jX4Vq3Xl2ZazLuzDkKQYwVt_3GtHGNQ=='},
  'RetryAttempts': 0},
 'Arn': 'arn:aws:dataexchange:us-east-1::data-sets/d0e9bd6148e8f14889980954017b0927',
 'AssetType': 'S3_SNAPSHOT',
 'CreatedAt': datetime.datetime(2020, 3, 27, 3, 43, 31, 511000, tzinfo=tzutc()),
 'Description': 'Data for 2020, updated weekly. ',
 'Id': 'd0e9bd6148e8f14889980954017b0927',
 'Name': 'US Imports CBP Automated

In [5]:
res = dx.list_data_set_revisions(DataSetId=data_set_id)

In [6]:
def get_all_assets(data_set_id, revision_id):
    assets = []
    res = dx.list_revision_assets(DataSetId=data_set_id,
                                  RevisionId=revision_id)
    next_token = res.get('NextToken')
    
    assets += res.get('Assets')
    while next_token:
        res = dx.list_revision_assets(DataSetId=data_set_id,
                                      RevisionId=revision_id,
                                      NextToken=next_token)
        assets += res.get('Assets')
        next_token = res.get('NextToken')
        
    return assets

In [7]:
res['Revisions'][0]['Arn'].split('/')[-1]

'83bdd5c3225a70b48acff63e86c74193'

In [8]:
assets = get_all_assets(data_set_id, res['Revisions'][0]['Arn'].split('/')[-1])

In [9]:
def export_assets(assets, bucket):
    
    asset_destinations = []

    for asset in assets:
        asset_destinations.append({
            "AssetId": asset.get('Id'),
            "Bucket": bucket,
            "Key": asset.get('Name')
        })

    job = dx.create_job(Type='EXPORT_ASSETS_TO_S3', Details={
        "ExportAssetsToS3": {
            "RevisionId": asset.get("RevisionId"), "DataSetId": asset.get("DataSetId"),
            "AssetDestinations": asset_destinations
        }
    })

    job_id = job.get('Id')
    dx.start_job(JobId=job_id)

    while True:
        job = dx.get_job(JobId=job_id)

        if job.get('State') == 'COMPLETED':
            break
        elif job.get('State') == 'ERROR':
            raise Exception("Job {} failed to complete - {}".format(
                job_id, job.get('Errors')[0].get('Message'))
            )

        time.sleep(60)

In [10]:
export_assets(assets, 'jacky-data-engineering-us-export')